In [36]:
from brainiak.isc import isc,bootstrap_isc,compute_summary_statistic
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [135]:
froot = '/data00/layerfMRI/Github_repo/layerfMRI/analyses/dual_ISC/'
os.listdir(froot)

['log_summary.csv',
 'fixing_log_file.Rmd',
 '.gitignore',
 'log_summary_FIXED.csv',
 'M_OR_S_JU_time_courses_thr100_bin10.csv',
 'dual_ISC_development_v7_ISC_data_furrr.Rmd',
 'data_native',
 'ANOVA_M_S_JU_time_courses_thr10_bin10.csv',
 'labels_juelich.csv',
 'M_OR_S_JU_time_courses_thr100_bin6.csv',
 'ANOVA_M_S_JU_time_courses_thr10_bin6.csv',
 'native_MNI_transformation',
 'OLE',
 'TC_4_lorenzo',
 'fixing_log_file.html',
 'dual_ISC_development_v7_ISC_data_furrr.html']

In [163]:
f = 'ANOVA_M_S_JU_time_courses_thr10_bin10.csv'

In [164]:
def r2numpy(rvec):
    return np.array(rvec[2:-1].split(', '),dtype = 'float')

def stdize(a):
    
    return (a - np.average(a)) / np.std(a)

In [165]:
# Read dataframe
df = pd.read_csv(froot + f)

# Transform timecourses into numpy arrays and standardize them
# df['tc_mean_unfolded'] = df['tc_mean_unfolded'].map(r2numpy).map(stdize)

In [166]:
# Get movie names
movie_names = {}
for c,gdf in df.groupby(['contrast']):
    movie_names[c] = gdf.sort_values(['muvi']).muvi.unique()

# Concatenate movie and task
cdf  = pd.DataFrame()
for [c,s,j,d],gdf in df.groupby(['contrast','sub','JU','D_bins']):

    gdf = gdf.sort_values(['muvi','run'])
    movie_list = gdf['muvi'].unique()

    try:
        same = (movie_names[c] == movie_list).all()
    except:
        same = False
    if not same:
        print(f'Error! contrast {c} subject {s} JU {j} D_bins {d}')
    
    concatenated = np.hstack(gdf['tc_mean_unfolded'])
    
    temp_df = gdf.iloc[0][['contrast','sub','task','JU','D_bins']]
    temp_df['tc_concatenated'] = concatenated

/home/deangelis/anaconda3/envs/brainiak/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


Error! contrast Motion subject sub_02 JU 27 D_bins 1
Error! contrast Motion subject sub_02 JU 27 D_bins 2
Error! contrast Motion subject sub_02 JU 27 D_bins 9
Error! contrast Motion subject sub_02 JU 35 D_bins 6
Error! contrast Motion subject sub_02 JU 35 D_bins 7
Error! contrast Motion subject sub_02 JU 35 D_bins 8
Error! contrast Motion subject sub_02 JU 45 D_bins 7
Error! contrast Motion subject sub_02 JU 51 D_bins 10
Error! contrast Motion subject sub_03 JU 35 D_bins 7
Error! contrast Motion subject sub_03 JU 35 D_bins 8
Error! contrast Motion subject sub_03 JU 45 D_bins 4
Error! contrast Motion subject sub_05 JU 27 D_bins 1
Error! contrast Motion subject sub_05 JU 27 D_bins 2
Error! contrast Motion subject sub_05 JU 27 D_bins 3
Error! contrast Motion subject sub_05 JU 27 D_bins 4
Error! contrast Motion subject sub_05 JU 27 D_bins 5
Error! contrast Motion subject sub_05 JU 27 D_bins 6
Error! contrast Motion subject sub_05 JU 27 D_bins 7
Error! contrast Motion subject sub_05 JU 27 D

Error! contrast Scrambled subject sub_12 JU 45 D_bins 5
Error! contrast Scrambled subject sub_12 JU 51 D_bins 5
Error! contrast Scrambled subject sub_12 JU 53 D_bins 2
Error! contrast Scrambled subject sub_12 JU 53 D_bins 4
Error! contrast Scrambled subject sub_14 JU 35 D_bins 10
Error! contrast Scrambled subject sub_14 JU 45 D_bins 3
Error! contrast Scrambled subject sub_14 JU 51 D_bins 4
Error! contrast Scrambled subject sub_14 JU 51 D_bins 5
Error! contrast Scrambled subject sub_14 JU 51 D_bins 7
Error! contrast Scrambled subject sub_14 JU 53 D_bins 1
Error! contrast Scrambled subject sub_14 JU 65 D_bins 7


In [162]:
df.loc[lambda d: (d['contrast']=='Scrambled')&(d['sub']=='sub_12')&(d['JU']==53)&(d['D_bins']==1)]

,contrast,sub,ses,task,run,muvi,start_TR,end_TR,dim4,ISC_flavour,JU,D_bins,tc_mean_unfolded
14884,Scrambled,sub_12,ses_02,4,2,S5cardD.avi,4,24,145,ANOVA_M_S,53,1,"c(0.00659841741268852, -0.763954349266334, -0...."
14914,Scrambled,sub_12,ses_02,4,2,S4buttonD.avi,27,43,145,ANOVA_M_S,53,1,"c(0.768275302549175, -1.3463455922931, 1.03924..."
14944,Scrambled,sub_12,ses_02,4,2,S2boatD.avi,69,91,145,ANOVA_M_S,53,1,"c(0.35168163690387, -0.894714365150443, -0.382..."


In [118]:
movie_names[c]

array(['M12letterD.avi', 'M13lightD.avi', 'M14nailD.avi', 'M15juiceD.avi',
       'M16pencilD.avi', 'M17pillowD.avi', 'M18removerD.avi',
       'M19sandwichD.avi', 'M1ballonD.avi', 'M21toastD.avi',
       'M22shirtD.avi', 'M2boatD.avi', 'M3breakfastD.avi',
       'M4buttonD.avi', 'M5cardD.avi', 'M6cigaretteD.avi',
       'M7flowersD.avi', 'M8frameD.avi'], dtype=object)

In [119]:
movie_list

array(['M13lightD.avi', 'M16pencilD.avi', 'M18removerD.avi',
       'M19sandwichD.avi', 'M22shirtD.avi', 'M2boatD.avi',
       'M4buttonD.avi', 'M5cardD.avi'], dtype=object)

In [76]:
concatenated

array([-0.10063118,  1.03533739,  0.28153692, -0.03453639,  1.31436405,
        0.17037449, -1.89025648, -1.68539773,  0.81093875,  0.09827017,
        0.38096231,  0.82067976,  0.2656863 , -2.37619736, -0.78376192,
       -0.6058662 , -0.33881884,  0.88065969,  1.03614526,  0.72051099,
        1.79451722, -0.62132233,  0.22196   ,  0.60343368,  0.24925965,
        0.11526566, -2.48202401,  0.09177084, -0.38265258,  1.12212246,
       -0.16033726, -0.55199332,  0.12320743,  0.44511459,  1.9338236 ,
       -0.44680525, -0.57336633, -1.48398288, -0.2320584 ,  1.89207467,
       -0.26014918, -1.14195165,  0.24497913, -0.50088574, -0.0452747 ,
        0.27875249,  2.60916573,  0.11399514, -1.08444325,  0.03749365,
       -0.63488371, -1.1603819 ,  0.65106098,  0.09457136, -0.8600558 ,
        0.49581068,  0.78730747,  2.37443858, -1.01156661, -0.09051985,
       -1.02316495, -1.31431737,  0.65436298, -0.24770316, -0.25772339,
       -0.36692438, -0.1110062 ,  1.72663434, -0.35436764,  0.64

In [78]:
gdf[['contrast','sub','ses','task','JU','D_bins']]

,contrast,sub,ses,task,JU,D_bins
62407,Scrambled,sub_14,ses_01,2,91,10
63707,Scrambled,sub_14,ses_01,2,91,10
64407,Scrambled,sub_14,ses_02,3,91,10
65797,Scrambled,sub_14,ses_02,3,91,10
62507,Scrambled,sub_14,ses_01,2,91,10
63607,Scrambled,sub_14,ses_01,2,91,10
62607,Scrambled,sub_14,ses_01,2,91,10
63507,Scrambled,sub_14,ses_01,2,91,10
64507,Scrambled,sub_14,ses_02,3,91,10
65687,Scrambled,sub_14,ses_02,3,91,10
